In [6]:
!pip install diffusers transformers accelerate safetensors opencv-python torch torchvision xformers


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from diffusers import (
    ControlNetModel,
    StableDiffusionXLControlNetPipeline,
    AutoencoderKL,
    EulerAncestralDiscreteScheduler
)


In [9]:
rooms = ["bathroom", "bedroom", "kitchen", "living_room"]

image_paths = []
prompts = []

for room in rooms:
    folder = os.path.join("/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata", room)
    for file in os.listdir(folder):
        if file.endswith((".jpg", ".png", ".jpeg")):
            image_paths.append(os.path.join(folder, file))
            prompts.append(f"{room} interior design, high quality")

In [10]:
def get_canny(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (1024, 1024))
    edges = cv2.Canny(img, 100, 200)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(edges)


In [12]:
controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0",
    torch_dtype=torch.float16
)

pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    torch_dtype=torch.float16,
).to("cuda")

pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [13]:
from torch.optim import AdamW

optimizer = AdamW(pipe.unet.parameters(), lr=1e-5)
transform = transforms.Compose([
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
])

EPOCHS = 3

for epoch in range(EPOCHS):
    for i, img_path in enumerate(image_paths):

        image = Image.open(img_path).convert("RGB")
        control = get_canny(img_path)

        image = transform(image).unsqueeze(0).to("cuda")
        control = transform(control).unsqueeze(0).to("cuda")

        prompt = prompts[i]

        out = pipe(
            prompt=prompt,
            image=image,
            control_image=control,
            num_inference_steps=10,
            guidance_scale=4.0,
            output_type="latent"
        )

        loss = out.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        print(f"Epoch {epoch} | Step {i} | Loss: {loss.item()}")

pipe.save_pretrained("/content/drive/MyDrive/SDXL_LoRA_Output")


In [1]:
from transformers import CLIPModel, CLIPProcessor

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

STYLE_LABELS = ["modern", "industrial", "boho", "luxury", "minimalist", "scandinavian"]

def get_style_type(style_img):
    inputs = clip_processor(
        text=STYLE_LABELS,
        images=style_img,
        return_tensors="pt",
        padding=True
    ).to("cuda")

    outputs = clip_model(**inputs)
    best = outputs.logits_per_image.argmax().item()
    return STYLE_LABELS[best]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [2]:
def generate_style(content_image_path, style_reference_path):

    content = Image.open(content_image_path).convert("RGB")
    style = Image.open(style_reference_path).convert("RGB")

    style_type = get_style_type(style)
    control = get_canny(content_image_path)

    final = pipe(
        prompt=f"{style_type} interior design, ultra realistic, 8k, high detail",
        image=content,
        control_image=control,
        guidance_scale=8.0,
        num_inference_steps=30,
    ).images[0]

    return final
